In [ ]:
from edf.pc_utils import draw_geometry, reconstruct_surface
from edf.data import PointCloud, SE3, TargetPoseDemo, DemoSequence, load_demos

import numpy as np
import yaml
import plotly as pl
import plotly.express as ple
import open3d as o3d

In [ ]:
demos  = load_demos("demo/test_demo/")
demo_seq: DemoSequence = demos[0]

In [ ]:
pcd = demo_seq[1].grasp_pc.to_pcd()
# draw_geometry(pcd)

In [ ]:
pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=20,
                                         std_ratio=2.0)

In [ ]:
if True:
    alpha = 0.015
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

if False:
    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(100)

    radii = [0.005, 0.01, 0.02, 0.04]
    mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pcd, o3d.utility.DoubleVector(radii))

    # o3d.visualization.draw_geometries([mesh], mesh_show_back_face=False)

if False:
    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))
    pcd.estimate_normals()
    pcd.orient_normals_consistent_tangent_plane(200)

    print('run Poisson surface reconstruction')
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd, depth=10)
    print(mesh)
    o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)

In [ ]:
mesh_in = mesh

mesh_out = mesh_in.filter_smooth_simple(number_of_iterations=5)
mesh_out.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh_out], mesh_show_back_face=True)